In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


In [2]:
# Загрузка данных
data = pd.read_csv('healthcare_dataset.csv')

# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 15

Типы данных по столбцам:
Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object


In [3]:
# Загрузка данных
data = pd.read_csv('healthcare_dataset.csv')

# Преобразование столбцов с датами в объекты datetime
data['Date of Admission'] = pd.to_datetime(data['Date of Admission'])
data['Discharge Date'] = pd.to_datetime(data['Discharge Date'])

# Вычисление продолжительности пребывания
data['Duration of Stay'] = (data['Discharge Date'] - data['Date of Admission']).dt.days

# Удаление исходных столбцов с датами
data.drop(['Date of Admission', 'Discharge Date'], axis=1, inplace=True)

# Отделение признаков и целевой переменной
X = data.drop(columns=['Medical Condition'])  # Предполагается, что 'Medical Condition' - целевая переменная
y = data['Medical Condition']

# Кодирование категориальных переменных
categorical_cols = ['Name', 'Gender', 'Blood Type', 'Doctor', 'Hospital', 'Insurance Provider', 'Admission Type', 'Medication', 'Test Results']
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Обработка отсутствующих значений и масштабирование
X.fillna(0, inplace=True)  # Заполнение отсутствующих значений нулями в качестве примера, выберите соответствующий метод
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Указание гиперпараметров и их значений для поиска
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': [0.1, 1, 10]}


# Создание объекта GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# Подгонка сетки по данным
grid_search.fit(X_train, y_train)

# Получение лучших параметров и лучшего оценщика
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Создание прогнозов с использованием лучшего оценщика
y_pred = best_svm.predict(X_test)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

# Определение отсутствующих классов в прогнозах
classes = np.unique(y_test)
predicted_classes = np.unique(y_pred)


Лучшие параметры: {'C': 1, 'gamma': 0.1, 'kernel': 'sigmoid'}
Точность модели: 0.1665
Отчет о классификации:
              precision    recall  f1-score   support

   Arthritis       0.15      0.19      0.17       319
      Asthma       0.17      0.20      0.18       332
      Cancer       0.20      0.18      0.19       363
    Diabetes       0.15      0.14      0.14       303
Hypertension       0.16      0.15      0.15       336
     Obesity       0.19      0.14      0.16       347

    accuracy                           0.17      2000
   macro avg       0.17      0.17      0.17      2000
weighted avg       0.17      0.17      0.17      2000



Программа загружает данные из CSV-файла , преобразует даты, рассчитывает длительность пребывания, убирает столбцы с датами, разделяет данные на признаки и целевую переменную, кодирует категориальные переменные, обрабатывает пропущенные значения и масштабирует данные, разбивает их на обучающие и тестовые наборы, определяет набор гиперпараметров для модели SVM, выполняет поиск наилучших параметров для SVM с помощью кросс-валидации и оценивает модель, выводя отчет о ее эффективности.